*This analysis goes over two very helpful techniques:*

1. The use of commands such as re.search as part of the RegEx expressions to extract values from strings.
2. Fuzzy String Matching.

Data Source: San_francisco_bikeshare, Big_Query, Google Cloud.

First: importing required environments.

In [ ]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz,process

Second: reading cvs dataset.

In [273]:
df = pd.read_csv('SFbikesharetrips.csv', encoding='latin1')

Third: examining data and variables.

In [274]:
df.head(5)

,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,member_birth_year,member_gender
0,2018-03-31 22:13:30 UTC,Post St at Kearny St,19,2018-03-31 22:34:35 UTC,Broderick St at Oak St,71,1021,37.788975,-122.403452,37.773063,-122.439078,1993.0,Female
1,2018-03-31 21:18:59 UTC,Valencia St at Cesar Chavez St,141,2018-03-31 21:24:24 UTC,Valencia St at 22nd St,133,1082,37.747998,-122.420219,37.755213,-122.420975,1992.0,Male
2,2018-03-31 20:24:12 UTC,Fulton St at Bancroft Way,247,2018-03-31 21:21:09 UTC,45th St at Manila,210,1038,37.867789,-122.265896,37.833294,-122.256224,1964.0,Male
3,2018-03-31 19:46:11 UTC,S. 4th St at San Carlos St,208,2018-03-31 19:51:35 UTC,San Salvador St at 9th St,317,2731,37.332836,-121.883912,37.333955,-121.877349,1992.0,Female
4,2018-03-31 17:18:07 UTC,Telegraph Ave at 27th St,179,2018-03-31 19:03:30 UTC,Shattuck Ave at Telegraph Ave,158,3208,37.816073,-122.267886,37.833279,-122.263490,NaN,NaN


In [275]:
df.shape

(1000, 13)

In [276]:
df.columns

Index([u'start_date', u'start_station_name', u'start_station_id', u'end_date',
       u'end_station_name', u'end_station_id', u'bike_number',
       u'start_station_latitude', u'start_station_longitude',
       u'end_station_latitude', u'end_station_longitude', u'member_birth_year',
       u'member_gender'],
      dtype='object')




Fourth: examining pick-up and drop-off addresses.

In [277]:
dfstartaddress = df['start_station_name']

In [278]:
dfstartaddress.head(5)

0              Post St at Kearny St
1    Valencia St at Cesar Chavez St
2         Fulton St at Bancroft Way
3        S. 4th St at San Carlos St
4          Telegraph Ave at 27th St
Name: start_station_name, dtype: object

Fifth: identifying main address types.

In [279]:
pick_up_address = []

for i in dfstartaddress:
    if re.search('St|Way|Ave|5th at Folsom|Broadway at Kearny', i):
        pick_up_address.append('Street Address')
    elif re.search('BART', i):
        pick_up_address.append('Bart Station Address')
    elif re.search('Park|Hall|Center|Square|Playground|Plaza|Public Market', i):
        pick_up_address.append('Public Place Address')
    else:
        pick_up_address.append('')

In [280]:
df['pick_up_address'] = pick_up_address

Sixth: making sure all address types have been identified.

In [281]:
df.isnull().sum()

start_date                  0
start_station_name          0
start_station_id            0
end_date                    0
end_station_name            0
end_station_id              0
bike_number                 0
start_station_latitude      0
start_station_longitude     0
end_station_latitude        0
end_station_longitude       0
member_birth_year          63
member_gender              63
pick_up_address             0
dtype: int64

In [282]:
pick_up_address

['Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Street Address',
 

Seventh: examining drop-off address.

In [283]:
dfendaddress = df['end_station_name']

In [284]:
dfendaddress.head(5)

0           Broderick St at Oak St
1           Valencia St at 22nd St
2                45th St at Manila
3        San Salvador St at 9th St
4    Shattuck Ave at Telegraph Ave
Name: end_station_name, dtype: object

Eighth: identifyng main address types.

In [285]:
drop_off_address = []

for a in dfendaddress:
    if re.search('St|Way|Ave|5th at Folsom|Broadway at Kearny', a):
        drop_off_address.append('Street Address')
    elif re.search('BART', a):
        drop_off_address.append('Bart Station Address')
    elif re.search('Park|Hall|Center|Square|Playground|Plaza|Public Market', a):
        drop_off_address.append('Public Place Address')
    else:
        drop_off_address.append('')

In [286]:
df['drop_off_address'] = drop_off_address

In [287]:
drop_off_address

['Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Public Place Address',
 'Street Address',
 'Public Place Address',
 'Public Place Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Street Address',
 'Stree

Nineth: making sure all addresses types have been identified.

In [288]:
df.isnull().sum()

start_date                  0
start_station_name          0
start_station_id            0
end_date                    0
end_station_name            0
end_station_id              0
bike_number                 0
start_station_latitude      0
start_station_longitude     0
end_station_latitude        0
end_station_longitude       0
member_birth_year          63
member_gender              63
pick_up_address             0
drop_off_address            0
dtype: int64

Tenth: FuzzyWuzzy,
    setting ratios with pick-up and drop-off addresses, through the 
    application of token set ratios    

In [301]:
def get_ratio(row):
    name = row['pick_up_address']
    namel = row['drop_off_address']
    return fuzz.token_set_ratio(name, namel)

Restricting the data to the pairs that exceeded a match score of 65.

In [304]:
df[df.apply(get_ratio, axis=1) > 65]

,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,member_birth_year,member_gender,pick_up_address,drop_off_address
0,2018-03-31 22:13:30 UTC,Post St at Kearny St,19,2018-03-31 22:34:35 UTC,Broderick St at Oak St,71,1021,37.788975,-122.403452,37.773063,-122.439078,1993.0,Female,Street Address,Street Address
1,2018-03-31 21:18:59 UTC,Valencia St at Cesar Chavez St,141,2018-03-31 21:24:24 UTC,Valencia St at 22nd St,133,1082,37.747998,-122.420219,37.755213,-122.420975,1992.0,Male,Street Address,Street Address
2,2018-03-31 20:24:12 UTC,Fulton St at Bancroft Way,247,2018-03-31 21:21:09 UTC,45th St at Manila,210,1038,37.867789,-122.265896,37.833294,-122.256224,1964.0,Male,Street Address,Street Address
3,2018-03-31 19:46:11 UTC,S. 4th St at San Carlos St,208,2018-03-31 19:51:35 UTC,San Salvador St at 9th St,317,2731,37.332836,-121.883912,37.333955,-121.877349,1992.0,Female,Street Address,Street Address
4,2018-03-31 17:18:07 UTC,Telegraph Ave at 27th St,179,2018-03-31 19:03:30 UTC,Shattuck Ave at Telegraph Ave,158,3208,37.816073,-122.267886,37.833279,-122.263490,NaN,NaN,Street Address,Street Address
5,2018-03-31 18:56:26 UTC,Lake Merritt BART Station,163,2018-03-31 19:02:00 UTC,2nd Ave at E 18th St,200,3091,37.797320,-122.265320,37.800214,-122.253810,NaN,NaN,Street Address,Street Address
6,2018-03-31 17:03:33 UTC,Mission Dolores Park,120,2018-03-31 17:15:27 UTC,Division St at Potrero Ave,89,3105,37.761420,-122.426435,37.769218,-122.407646,1988.0,Female,Public Place Address,Street Address
7,2018-03-31 16:36:59 UTC,Irwin St at 8th St,102,2018-03-31 16:58:35 UTC,16th St Mission BART Station 2,223,3654,37.766883,-122.399579,37.764765,-122.420091,1990.0,Male,Street Address,Street Address
8,2018-03-31 16:36:50 UTC,Irwin St at 8th St,102,2018-03-31 16:57:43 UTC,16th St Mission BART,108,2441,37.766883,-122.399579,37.764710,-122.419957,1988.0,Male,Street Address,Street Address
9,2018-03-31 16:12:25 UTC,Delmas Ave and San Fernando St,283,2018-03-31 16:20:38 UTC,Paseo De San Antonio at 2nd St,311,2704,37.330264,-121.897702,37.333798,-121.886943,1974.0,Male,Street Address,Street Address


In [305]:
len(df[df.apply(get_ratio, axis=1) > 65])/len(df)

1

Eleventh: exporting dataset to csv.

In [310]:
df.to_csv(r'SFBiketrips.cvs', index = False)